![title](images\scala-logo.png)

# RDD

Conjunto de Datos Distribuidos Resilientes

¿Que es un conjunto de datos?
Un conjunto de datos es basicamente una coleccion de datos; puede ser una lista de strings, una lista de enteros e incluso
el numero de filas de una base de datos relacional. Pueden contener cualquier tipo de objetos, incluidas clases definidas por el usuario
Un RDD es simplemente una compilacion alrededor de un gran conjunto de datos. En Spark todo el trabajo esta expresado ya sea 
creando nuevas RDDs, transformando RDDs existentes, o ejecutando operaciones en RDDs para calcular un resultado.
Spark se encarga de distribuir automaticamente los datos contenidos en RDDs a travez de tu cluster y paralelizara las opreaciones que realices en ellas.

  ¿Que podemos hacer con los RDDs?
1. Transformaciones: Aplican basicamente algunas funciones a los datos de un RDD para crear un nuevo RDD
  Una de las transformaciones mas comunes es filterla cual genera un nuevo RDD que contiene un subconjunto de datos de un RDD     existente
  
2. Accioes: Calcula un resultado basado en un RDD
   Una de las acciones mas populares es first, la cual retornara el primer elemento de un RDD existente

# Flujo de Trabajo de Spark RDD

- Genererará RDDs iniciales a partir de datos externos
- Aplicara transformaciones
- Iniciara acciones 

# Creacion de RDDs

La creacion de RDDs se puede dar de dos maneras:
1. Cargando una BD externa
2. Paralelizando una coleccion en el programa driver

In [ ]:
// Haciendo uso del primer metodo 
val inputInt = List(1,2,3,4,5)
val intRDD = sc.parallelize(inputInt)

Una forma comun de trabajar con RDDs es:
Carga los RDDs desde un almancenamiento externo usando el metodo textfile en SparkContext

# Transformaciones Map y Filter en Apache Spark

Son operaciones sire RDDs las cuales generan como resultado un nuevo RDD.
Las dos transformaciones mas comunes son filter y map

## Transformacion Filter

Se ejecuta como una funcion y genera un RDD formado por aquellos elementos seleccionados sobre los cuales se ejecuta la funcion filter

Puede ser usada para eliminar algunas filas invalidas, para limpiar el RDD input o solo para conseguir un sobconjunto del RDD basado en la funcion filtro

## Transformacion Map

Se ejecuta como una funcion sobre cada elemento del RDD input y esta funcion genera un nuevo valor para cada elemento los cuales se almacena en el RDD Resultante 

Puede ser usada para crear solicitudes HTTP para cada URL contenida en nuestro RDD input, o puede ser usada para calcular la raiz cuadrada de cada numero

El tipo de valores como resultado de ejecutar la funcion Map no son necesariamente los mismos del RDD input

### Ejercicio:

Cree un programa Spark para leer los datos del aeropuerto desde / airports.text, encuentre todos los aeropuertos ubicados en Estados Unidos y enviar el nombre del aeropuerto y el nombre de la ciudad a / airports_in_usa.text. Cada fila del archivo de entrada contiene las siguientes columnas:  Identificación del aeropuerto, nombre del aeropuerto, ciudad principal a la que sirve el aeropuerto, país donde se encuentra el aeropuerto, código IATA / FAA,  Código OACI, latitud, longitud, altitud, zona horaria, horario de verano, zona horaria en formato Olson.

### Solucion:

In [ ]:
import org.apache.spark._

val airports = sc.textFile("input/airports.text")
val airportsInUSA = airports.filter(line => line.split(",")(3) == "\"United States\"")

val airportsNameAndCityNames = airportsInUSA.map(line => {
      val splits = line.split(",")
      splits(1) + ", " + splits(2)
    })
    airportsNameAndCityNames.saveAsTextFile("out/airports_in_usa.text")


### Ejercicio

Cree un programa Spark para leer los datos del aeropuerto desde / airports.text, encuentre todos los aeropuertos cuya latitud sea mayor que 40. Luego, envíe el nombre del aeropuerto y la latitud del aeropuerto a / airports_by_latitude.text.
Cada fila del archivo de entrada contiene las siguientes columnas: Identificación del aeropuerto, nombre del aeropuerto, ciudad principal a la que sirve el aeropuerto, país donde se encuentra el aeropuerto, código IATA / FAA, Código OACI, latitud, longitud, altitud, zona horaria, horario de verano, zona horaria en formato Olson

### Solucion

In [ ]:
import org.apache.spark._

val airports = sc.textFile("input/airports.text")
val airportsInUSA = airports.filter(line => line.split(",")(6).toFloat > 40)

val airportsNameAndCityNames = airportsInUSA.map(line => {
val splits = line.split(",")
     splits(1) + ", " + splits(6)
    })
 //   airportsNameAndCityNames.saveAsTextFile("out/airports_by_latitude.text")

# Transformacion FlatMap

Genera un nuevo RDD al primero aplicar una funcion a todos los elementos de este RDD y luego aplanando los resultados

Ejercicio:
WordCount

In [ ]:
import org.apache.log4j.Level
import org.apache.log4j.Logger
import org.apache.spark.SparkConf
import org.apache.spark._  
   
   Logger.getLogger("org").setLevel(Level.ERROR)

    val lines = sc.textFile("input/word_count.text")
    val words = lines.flatMap(line => line.split(" "))

    val wordCounts = words.countByValue()
   // countByValue() nos dara como resutlado la cantidad de cada valor unico del RDD 
    for ((word, count) <- wordCounts) println(word + " : " + count)

## Operaciones Set

Las operaciones Set son realizadas en un RDD
 - Sample
 - Distinct

### Sample

- La operacion Sample creara un ejemplo al azar a partir de un RDD
- Muy practico para realizar pruebas

### Distinct

Provee un nuevo RDD conteniendo los elementos distinct en este RDD

- La transformacion retornara unicamente filas que sean distintas a partir de un RDD input
- La operacion distinct es muy costosa por que requiere combinar todos los datos a travez de particiones para asegurar que recibiremos una sola copia de cada elemento

#### Operaciones Set que se ejecutan sobre dos RDDs y producen un RDD resultante

- Union
- Intersection
- Substract
- Cartesian product

#### Operacion de conjuntos

![title](images\operaciones-conjuntos.png)

Producto cartesiano

Las transformaciones cartesian genera todos los posibles emparejamientos de A y B donde A y B son fuentes que corresponden a cada RDD

![title](images\product_cartesian.gif)

#### Ejemplo

El archivo "in / nasa_19950701.tsv" contiene 10000 líneas de registro de uno de los servidores apache de la NASA para el 1 de julio de 1995.
El archivo "in / nasa_19950801.tsv" contiene 10000 líneas de registro para el 1 de agosto de 1995
Cree un programa Spark para generar un nuevo RDD que contenga las líneas de registro tanto del 1 de julio como del 1 de agosto,
tome una muestra de 0.1 de esas líneas de registro y guárdela en el archivo "out / sample_nasa_logs.tsv".
 Tenga en cuenta que los archivos de registro originales contienen las siguientes líneas de encabezado. host logname tiempo método url respuesta bytes
 Asegúrese de que las líneas principales se eliminen en el RDD resultante.

#### Solucion:

In [ ]:
import org.apache.spark._ 

val julyFirstLogs = sc.textFile("input/nasa_19950701.tsv")
val augustFirstLogs = sc.textFile("input/nasa_19950801.tsv")
// Realizamos la union de los dos archivos de texto
val aggregatedLogLines = julyFirstLogs.union(augustFirstLogs)

// Imprimimos las 10  primeras filas del RDD
aggregatedLogLines.take(10).foreach(println)

//Contamos la cantidad de registros del RDD
aggregatedLogLines.count()

// Podemos tambien borrar los encabezados del RDD
aggregatedLogLines.collect().drop(1)

// Limpiamos quitando los encabezados
//val cleanLogLines = aggregatedLogLines.filter(line => isNotHeader(line))

//val sample = cleanLogLines.sample(withReplacement = true, fraction = 0.1)

// Por ultimo guardamos el RRD resultante como archivo de texto
//sample.saveAsTextFile("out/sample_nasa_logs.csv")
// }

//def isNotHeader(line: String): Boolean = !(line.startsWith("host") && line.contains("bytes"))


# Acciones

- Las acciones son las que generan un valor final al programa driver o mantienen los datos de un sistema de almacenamiento interno
- LAs acciones obligaran la evaluacion de las transformaciones requeridas para el RDD al que  fueron llamadas

Acciones mas Populares:
- collect
- count
- countByValue
- take
- saveAsTextFile
- reduce

### Collect

La operacion collect obtiene el RDD entero y lo genera en el programa driver en forma de una coleccion regular o valor

- Si tienes un string RDD, cuando utilizas una accion en el, conseguiras una lista de string
- Es muy util si el programa spark tiene RDDs filtrados por debajo de un tamaño relativamente pequeño y deseas lidiar con ellos localmente
- La base de datos entera debe ajustarse en la memoria en una sola maquina ya que todo lo que necesitas es ser copiada la driver cuando la accion collect es utilizada. Entonces la collect NO deberia ser usada en grandes bases de datos
- La operacion Collect es ampliamente usada en pruebas unitarias para comparar el valor de nuestro RDD cn nuestro resultado esperado, siempre y cuando todo el contenido del RDD se ajuste en la memoria

#### Ejemplo:

In [ ]:
val inputWords = List("spark", "hadoop", "spark", "hive", "pig", "cassandra", "hadoop")
val wordRdd = sc.parallelize(inputWords)

val words = wordRdd.collect()

for (word <- words) println(word)

### Count & ContByValue

- Si quieres contar el numero de filas en un RDD, la operacion count es una manera rapida de hacerlo. Esta genera el contero de los elementos

- countByValue buscara valores unicos en cada fila de tu RDD y generara un mapeo de cada valor unico con su conteo

#### Ejemplo

In [ ]:
val inputWords = List("spark", "hadoop", "spark", "hive", "pig", "cassandra", "hadoop")
val wordRdd = sc.parallelize(inputWords)
println("Count: " + wordRdd.count())

println("------------------------------------")

val wordCountByValue = wordRdd.countByValue()
println("CountByValue:")
for ((word, count) <- wordCountByValue) println(word + " : " + count)


### Take

- La accion take toma n cantidad de elemenetos de un RDD
- La operacion take puede ser muy util para realizar pruebas unitarias y uyna rapida depuracion
- take genera n elementos apartir de un RDD y esta intentara reducir el numero de particiones a las que accede, entonces es posible que la operacion take termine dandonos una coleccion aleatoria y no necesariamente generar los elementos en el orden que podriamos esperear

#### Ejemplo

In [ ]:
val inputWords = List("spark", "hadoop", "spark", "hive", "pig", "cassandra", "hadoop", "hive", "Hbase", "Sqoop")
val wordRdd = sc.parallelize(inputWords)

val words = wordRdd.take(3)
for (word <- words) println(word)

### SaveAsTextFile

SaveAsTextFile puede ser usado para escribi datos de salida a sistemas de almacenamiento distribuidos como HDFS o Amazon S3, o inclusive a archivos del sistema local

### Reduce

- Reduce toma una funcion que opera sobre dos elementos que hay en el RDD de entrada y devuelve un Nuevo elemento del mismo tipo. Esto reduce los elementos de este RDD usando la funcion binaria especificada.
- Esta funcion produce el mismo resultado cuando se aplica de forma repetitiva en el mismo conjunto de datos RDD y se reduce a un solo valor
- Con la operacion Reduce podemos facilmente sumar todos los elementos de un RDD, contar el numero total de elementos o realizar otros tipos de operaciones:

#### Ejemplo:

In [ ]:
val inputIntegers = List(1, 2, 3, 4, 5)
val integerRdd = sc.parallelize(inputIntegers)

val product = integerRdd.reduce((x, y) => x * y)
println("product is :" + product)

#### Ejercicio:

Cree un programa Spark para leer los primeros 100 números primos en / prime_nums.text, imprime la suma de esos números a la consola. Cada fila del archivo de entrada contiene 10 números primos separados por espacios.

#### Solucion:

In [ ]:
val lines = sc.textFile("input/prime_nums.text")

val numbers = lines.flatMap(line => line.split("\\s+"))

val validNumbers = numbers.filter(number => !number.isEmpty)

val intNumbers = validNumbers.map(number => number.toInt)

println("Sum is: " + intNumbers.reduce((x, y) => x + y))

## Aspectos importantes de RDDs

- RDDs son distribuidos: se divide en partes llamadas particiones y cada particion se divide a traves del cluster
- Es proceso de particionado se realiza automaticamente por Spark, lo que no necesitas preocuparte por todos los detalles acerca de como se dividen sus datos a traves del cluster.
- Los RDD son inmutables: No pueden ser cambiados despues de ser creados
- Los RDD son resilientes: son una funcion deterministica de sus datos de entrada. Esto mas inmutabilidad tambien significa que ciertas partes de los RDDs se pueden crecrear en cualquier momento.
- En caso de que un nodo del cluster se caiga, Spark puede recuperar las partes de los RDDs de los datos de entrada y recogerlos desde donde quedaron.
- Son Tolerantes a fallos

##### Resumen: Las tranasformaciones generan RDDs, mientras que las acciones generan totalmente algun otro tipo de dato

### Almacenamiento en Cache y Persistencia (Caching y Persistence)

#### Persistence

- Algunas veces nos gustaria utilizar acciones sobre el mismo RDD muchas veces
- Si hacemos esto sin preocupacion, los RDD y todos sus componentes seran recalculados cada vez que una accion sea utilizada sobre un RDD
- Esto puede ser muy costoso, especialmente por algunos algoritmos iterativos los cuales utilizan acciones en la misma base de datos varias veces.
- Si quieres reusar un RDD en multiples acciones, puedes tambien pedir a Spark que persista mediante el uso del metodo persist() sobre el RDD.
- Cuando persistes un RDD, la primera vez es calculando en una accion este se mantendra en la memoria a traves de los nodos
- Permite que las acciones futuras sean mas rapidas

#### Ejemplo: 

In [ ]:
import org.apache.spark.storage.StorageLevel

val inputIntegers = List(1, 2, 3, 4, 5)
val integerRdd = sc.parallelize(inputIntegers)

integerRdd.persist(StorageLevel.MEMORY_ONLY)

integerRdd.reduce((x, y) => x * y)
integerRdd.count()

//RDD.cache() = RDD.persist(MEMORY_ONLY)

# Arquitectura Spark (Maestro - Esclavo)


![title](images\spark-architecture.png)

# Arquitectura de la computación en clúster

![title](images\spark-architecture-II.png)

# Componentes Spark

![title](images\components-spark.png)

Spark Core proporciona las APIs necesarias para la manitpulacion de estas colecciones y permite la ejecucion de modelos, Tareas y Almacenamiento en cache